### **Función de Preprocesado de Texto**

clean_text: Limpia el texto eliminando signos de puntuación y convirtiendo todo a minúsculas para uniformidad.
tokenize_text: Convierte el texto limpio en tokens (palabras individuales) utilizando la tokenización de NLTK.
remove_stopwords: Elimina las palabras comunes que suelen ser poco informativas para los modelos de NLP.
lemmatize_words: Reduce las palabras a su forma base o lema, lo que ayuda a consolidar diferentes formas de una palabra para que sean tratadas como una sola entidad.
preprocess_text: Integra todas las funciones anteriores en una secuencia de operaciones que prepara el texto completamente.

In [1]:
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk

# Descargar recursos necesarios de NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def clean_text(text):
    """Limpiar el texto removiendo puntuación y convirtiendo el texto a minúsculas."""
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar puntuación
    text = re.sub(r'[^\w\s]', '', text)
    return text

def tokenize_text(text):
    """Tokenizar el texto en palabras individuales."""
    return word_tokenize(text)

def remove_stopwords(tokens):
    """Eliminar stopwords del texto tokenizado."""
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    return filtered_tokens

def lemmatize_words(tokens):
    """Aplicar lematización a los tokens para reducirlos a su forma base o de lema."""
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmatized_tokens

def preprocess_text(text):
    """Función completa de preprocesamiento que integra todos los pasos."""
    text = clean_text(text)
    tokens = tokenize_text(text)
    tokens = remove_stopwords(tokens)
    lemmatized_tokens = lemmatize_words(tokens)
    # Unir los tokens en una cadena para análisis posterior
    preprocessed_text = ' '.join(lemmatized_tokens)
    return preprocessed_text



[nltk_data] Downloading package punkt to
[nltk_data]     /Users/otgerpeidro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/otgerpeidro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/otgerpeidro/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/otgerpeidro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/otgerpeidro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/otgerpeidro/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

**Descargar los Datasets**

Descargamos de nuevo los datasets ya que estamos en otro notebook

In [3]:
import requests

def download_file(url, filename):
    response = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(response.content)

download_file("https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_2023/raw/review_categories/Software.jsonl.gz", "Software.jsonl.gz")
download_file("https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_2023/raw/review_categories/Digital_Music.jsonl.gz", "Digital_Music.jsonl.gz")

**Funciones para Cargar y Preprocesar Datos**

In [4]:
import gzip
import json
import pandas as pd

def load_data(file_name, n_rows=1000):
    data = []
    with gzip.open(file_name, 'rt', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= n_rows:
                break
            data.append(json.loads(line))
    return pd.DataFrame(data)

software_data = load_data('Software.jsonl.gz', 1000)
digital_music_data = load_data('Digital_Music.jsonl.gz', 1000)

In [5]:
# Aplicar el preprocesamiento
software_data['preprocessed_text'] = software_data['text'].apply(preprocess_text)
digital_music_data['preprocessed_text'] = digital_music_data['text'].apply(preprocess_text)

In [7]:
# Mostrar algunos resultados
print(software_data[['text', 'preprocessed_text']].head())
print(digital_music_data[['text', 'preprocessed_text']].head())

                                                text  \
0                                 mcaffee IS malware   
1  I love playing tapped out because it is fun to...   
2  I love this flashlight app!  It really illumin...   
3                           One of my favorite games   
4  Cute game. I am not that good at it but my kid...   

                                   preprocessed_text  
0                                    mcaffee malware  
1  love playing tapped fun watch town grow earnin...  
2  love flashlight app really illuminates dark co...  
3                                  one favorite game  
4               cute game good kid love nik wallenda  
                                                text  \
0  If i had a dollar for how many times I have pl...   
1  awesome sound - cant wait to see them in perso...   
2  This is a great cd. Good music and plays well....   
3  These are not real German singers, they have a...   
4  I first heard this playing in a Nagoya shop an... 

In [8]:
# Visualizar los cambios
for index, row in digital_music_data.head(5).iterrows():
    print("Original:", row['text'])
    print("Preprocesado:", row['preprocessed_text'])
    print("---")


Original: If i had a dollar for how many times I have played this cd and how many times I have asked Alexa to play it, I would be rich. Love this singer along with the Black Pumas. Finding a lot of new music that I like a lot on amazon. Try new things.
Preprocesado: dollar many time played cd many time asked alexa play would rich love singer along black puma finding lot new music like lot amazon try new thing
---
Original: awesome sound - cant wait to see them in person - always miss them when they are in town !
Preprocesado: awesome sound cant wait see person always miss town
---
Original: This is a great cd. Good music and plays well. Seller responded back very quicky and  received it within 3 days
Preprocesado: great cd good music play well seller responded back quicky received within 3 day
---
Original: These are not real German singers, they have accents. It is nothing what they advertised it. Music stinks.
Preprocesado: real german singer accent nothing advertised music stink
---

In [9]:
# Calcular tamaño del vocabulario
from collections import Counter

def vocab_size(data):
    all_words = ' '.join(data).split()
    vocabulary = Counter(all_words)
    return len(vocabulary)

original_vocab_size = vocab_size(digital_music_data['text'])
processed_vocab_size = vocab_size(digital_music_data['preprocessed_text'])
print("Vocabulario original:", original_vocab_size)
print("Vocabulario después del preprocesamiento:", processed_vocab_size)


Vocabulario original: 13721
Vocabulario después del preprocesamiento: 8240


In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

**Preparación de Datos**

In [11]:

software_data['sentiment'] = software_data['rating'].apply(lambda x: 1 if x >= 3 else 0)
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(software_data['preprocessed_text'], software_data['sentiment'], test_size=0.2, random_state=42)

vectorizer_s = CountVectorizer(max_features=1000)
X_train_vec_s = vectorizer_s.fit_transform(X_train_s)
X_test_vec_s = vectorizer_s.transform(X_test_s)

# Modelos
logistic_model_s = LogisticRegression(random_state=42)
logistic_model_s.fit(X_train_vec_s, y_train_s)
decision_tree_model_s = DecisionTreeClassifier(random_state=42)
decision_tree_model_s.fit(X_train_vec_s, y_train_s)

# Evaluación
y_pred_logistic_s = logistic_model_s.predict(X_test_vec_s)
y_pred_tree_s = decision_tree_model_s.predict(X_test_vec_s)
print("Software - Logistic Regression:\n", classification_report(y_test_s, y_pred_logistic_s))
print("Software - Decision Tree:\n", classification_report(y_test_s, y_pred_tree_s))


Software - Logistic Regression:
               precision    recall  f1-score   support

           0       0.59      0.41      0.48        54
           1       0.80      0.90      0.85       146

    accuracy                           0.77       200
   macro avg       0.70      0.65      0.67       200
weighted avg       0.75      0.77      0.75       200

Software - Decision Tree:
               precision    recall  f1-score   support

           0       0.48      0.59      0.53        54
           1       0.83      0.76      0.80       146

    accuracy                           0.71       200
   macro avg       0.66      0.68      0.66       200
weighted avg       0.74      0.71      0.72       200



In [12]:
digital_music_data['sentiment'] = digital_music_data['rating'].apply(lambda x: 1 if x >= 3 else 0)
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(digital_music_data['preprocessed_text'], digital_music_data['sentiment'], test_size=0.2, random_state=42)

vectorizer_d = CountVectorizer(max_features=1000)
X_train_vec_d = vectorizer_d.fit_transform(X_train_d)
X_test_vec_d = vectorizer_d.transform(X_test_d)

# Modelos
logistic_model_d = LogisticRegression(random_state=42)
logistic_model_d.fit(X_train_vec_d, y_train_d)
decision_tree_model_d = DecisionTreeClassifier(random_state=42)
decision_tree_model_d.fit(X_train_vec_d, y_train_d)

# Evaluación
y_pred_logistic_d = logistic_model_d.predict(X_test_vec_d)
y_pred_tree_d = decision_tree_model_d.predict(X_test_vec_d)
print("Digital Music - Logistic Regression:\n", classification_report(y_test_d, y_pred_logistic_d))
print("Digital Music - Decision Tree:\n", classification_report(y_test_d, y_pred_tree_d))


Digital Music - Logistic Regression:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.95      0.99      0.97       190

    accuracy                           0.94       200
   macro avg       0.47      0.50      0.49       200
weighted avg       0.90      0.94      0.92       200

Digital Music - Decision Tree:
               precision    recall  f1-score   support

           0       0.15      0.20      0.17        10
           1       0.96      0.94      0.95       190

    accuracy                           0.91       200
   macro avg       0.56      0.57      0.56       200
weighted avg       0.92      0.91      0.91       200



**Justificación de Parámetros y Modelos**

**CountVectorizer**:
max_features=1000: Limitamos el número de características para manejar mejor la complejidad y el rendimiento del modelo, especialmente útil cuando manejamos grandes volúmenes de texto.

**Regresión Logística**: es un modelo lineal simple y efectivo para clasificación binaria, ideal para líneas base.<br> **Árbol de Decisión**: proporciona un modelo más complejo que puede capturar relaciones no lineales, pero puede sobreajustarse si no se configura correctamente.

Observemos los resultados:

1. Digital Music - Logistic Regression:

+ Precisión muy alta para la clase 1 (0.95), pero incapaz de identificar correctamente ningún caso de la clase 0 (precisión de 0.00).
+ Puntaje F1 total muy alto (0.97 para clase 1), pero el modelo falla completamente en la clase 0.
+ Alta precisión general (0.94), pero esto es principalmente porque la mayoría de las muestras son de la clase 1.

2. Digital Music - Decision Tree:

+ Mejor rendimiento general en la clasificación de ambas clases comparado con la regresión logística en el mismo conjunto de datos.
+ Precisión moderada para la clase 0 (0.15) y alta para la clase 1 (0.96).
+ Puntaje F1 general equilibrado (0.95 para clase 1 y 0.17 para clase 0).
+ La precisión general es algo menor (0.91) pero con mejor equilibrio entre clases.

3. Software - Logistic Regression:

+ Precisión y recall equilibrados para la clase 0 comparados con el modelo de árbol de decisiones.
+ Alto recall para la clase 1 (0.90) con buena precisión (0.80).
+ Puntaje F1 general bueno para ambas clases y precisión general decente (0.77).

4. Software - Decision Tree:

+ Mejor rendimiento en recall para la clase 0 comparado con la regresión logística, pero menor precisión.
+ Precisión y recall razonablemente equilibrados para ambas clases.
+ Puntaje F1 y precisión general algo menores que la regresión logística.

**Elección del Modelo** <br>
Para el conjunto de datos Digital Music, el árbol de decisiones parece ser la mejor opción, ya que aunque su precisión general es ligeramente inferior a la de la regresión logística, muestra un mejor balance en la identificación de ambas clases, lo cual es crucial para un modelo útil en escenarios del mundo real donde ambas clases son importantes.
<br>
Para el conjunto de datos Software, la regresión logística es preferible ya que, aunque el árbol de decisiones tiene un recall ligeramente superior para la clase 0, la regresión logística proporciona una mejor precisión general y un mejor balance de precisión y recall, especialmente para la clase 1 que tiene más muestras.

In [13]:
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

### **Calcular Métricas de Evaluación**

**Para Digital Music (Árbol de Decisión)**

In [14]:
print("Evaluación del Árbol de Decisión para Digital Music:")
print(classification_report(y_test_d, y_pred_tree_d))

# Calculando la matriz de confusión
conf_matrix_d = confusion_matrix(y_test_d, y_pred_tree_d)
print("Matriz de Confusión:\n", conf_matrix_d)

# Calculando la precisión general
accuracy_d = accuracy_score(y_test_d, y_pred_tree_d)
print("Precisión General:", accuracy_d)

Evaluación del Árbol de Decisión para Digital Music:
              precision    recall  f1-score   support

           0       0.15      0.20      0.17        10
           1       0.96      0.94      0.95       190

    accuracy                           0.91       200
   macro avg       0.56      0.57      0.56       200
weighted avg       0.92      0.91      0.91       200

Matriz de Confusión:
 [[  2   8]
 [ 11 179]]
Precisión General: 0.905


**Para Digital Music (Árbol de Decisión)**

In [15]:
print("Evaluación de la Regresión Logística para Software:")
print(classification_report(y_test_s, y_pred_logistic_s))

# Calculando la matriz de confusión
conf_matrix_s = confusion_matrix(y_test_s, y_pred_logistic_s)
print("Matriz de Confusión:\n", conf_matrix_s)

# Calculando la precisión general
accuracy_s = accuracy_score(y_test_s, y_pred_logistic_s)
print("Precisión General:", accuracy_s)

Evaluación de la Regresión Logística para Software:
              precision    recall  f1-score   support

           0       0.59      0.41      0.48        54
           1       0.80      0.90      0.85       146

    accuracy                           0.77       200
   macro avg       0.70      0.65      0.67       200
weighted avg       0.75      0.77      0.75       200

Matriz de Confusión:
 [[ 22  32]
 [ 15 131]]
Precisión General: 0.765


### Análisis de los Resultados

#### Digital Music - Árbol de Decisión
- **Desempeño General**: El modelo alcanza una precisión general del 90.5%, lo cual es bastante alto. Sin embargo, la efectividad del modelo varía significativamente entre las clases.
- **Clase Minoritaria (0)**: La precisión y el recall para la clase minoritaria (0) son bastante bajos (15% de precisión y 20% de recall), lo que indica que el modelo tiene dificultades para identificar correctamente los casos negativos, aunque supera ligeramente el rendimiento del modelo de regresión logística en este aspecto.
- **Clase Mayoritaria (1)**: El modelo muestra una alta precisión (96%) y un buen recall (94%) para la clase mayoritaria, lo que indica que es bastante efectivo para identificar los casos positivos.
- **Matriz de Confusión**: La matriz muestra que el modelo predijo correctamente 179 de 190 casos positivos, pero sólo 2 de 10 casos negativos, reforzando la observación de que el modelo está sesgado hacia la clase mayoritaria.

#### Software - Regresión Logística
- **Desempeño General**: El modelo tiene una precisión general del 76.5%. Aunque es aceptable, hay margen de mejora, especialmente en la identificación de clases negativas.
- **Clase Minoritaria (0)**: El modelo muestra una precisión del 59% y un recall del 41% para la clase 0, lo que sugiere una capacidad moderada para identificar críticas negativas, pero sigue siendo relativamente bajo.
- **Clase Mayoritaria (1)**: La precisión y el recall son relativamente altos para la clase mayoritaria (80% y 90% respectivamente), lo que indica que el modelo es efectivo para identificar reseñas positivas.
- **Matriz de Confusión**: El modelo identificó correctamente 131 de 146 casos positivos pero tuvo un desempeño menos robusto en los negativos, con 22 aciertos de 54.

### Conclusiones Finales

- **Equilibrio de Clases**: Ambos modelos muestran un desempeño superior en la identificación de la clase mayoritaria en comparación con la minoritaria. Esto es un reflejo común del desbalance de clases en los conjuntos de datos. Estrategias como el reequilibrio de clases, el ajuste de costos o la exploración de modelos más complejos podrían ayudar a mejorar la identificación de la clase minoritaria.
- **Elección de Modelos**: La Regresión Logística, aunque más simple, demostró ser efectiva para el conjunto de datos de Software, proporcionando un buen equilibrio entre precisión y recall para la clase mayoritaria y un desempeño razonable para la minoritaria. El Árbol de Decisión fue más efectivo para Digital Music en términos de equilibrio general, aunque su precisión para la clase minoritaria aún necesita mejora.